In [7]:
from atlassian import Jira
from bidict import bidict

# url = 'https://tangramcare.atlassian.net/'
# username = 'awaz@ownedoutcomes.com'
# api_token = '6Pu9J7zSN4wBqwqgSREsAE08'

jira_info = {
    'url': 'https://tangramcare.atlassian.net/',
    'username': 'awaz@ownedoutcomes.com',
    'api_token': '6Pu9J7zSN4wBqwqgSREsAE08',
}

# jira_info = {
#     'url': 'https://awaz.atlassian.net/',
#     'username': 'wazartur@gmail.com',
#     'api_token': 'typYw7sWGOPzOLvtq5Z91BDC',
# }

# jira standard fields config
jira_standard_fields = bidict({
    'Summary': 'summary',
    'Epic': 'customfield_10008',
    'Description': 'description',
    'Project': 'project',
    'Type': 'issuetype',
    'Priority': 'priority',
    'Labels': 'labels',
    'Status': 'status',
    'Creator': 'creator',
    'Assignee': 'assignee',
    'Parent': 'parent',
})

jira = Jira(
            url=jira_info['url'],
            username=jira_info['username'],
            password=jira_info['api_token'],
            cloud=True)

In [61]:
issues = jira.jql('project=tt', fields=list(o2_pp_fields.inv.keys()), limit=1000)
print(str(issues['maxResults']))
print(str(len(issues['issues'])))

100
0


In [14]:
from openpyxl import Workbook

workbook = Workbook()
sheet = workbook.active

sheet["A1"].value = "Design document:"
sheet["C1"].value = "Link"
sheet["C1"].hyperlink = "https://docs.google.com/document/d/1B5RlsVSZdiuKTVAqhrn7xClewqhzrxhliUZd_yIHUBg/edit?usp=sharing"
sheet["C1"].style = "Hyperlink"

sheet.merge_cells('A3:A4')
sheet['A3'].value = 'WBS'

sheet.merge_cells('B3:B4')
sheet['B3'].value = 'Name'

sheet.merge_cells('C3:C4')
sheet['C3'].value = 'Description'

sheet.merge_cells('D3:D4')
sheet['D3'].value = 'Depends on'

sheet.merge_cells('E3:G3')
sheet['E3'].value = 'Estimated development time [h]'
sheet['E4'].value = 'Designer'
sheet['F4'].value = 'Reviewer'
sheet['G4'].value = 'PM'

sheet.merge_cells('H3:H4')
sheet['H3'].value = 'Assigned developer'

sheet.merge_cells('I3:J3')
sheet['I3'].value = 'Estimated testing time [h]'
sheet['I4'].value = 'Tester'
sheet['J4'].value = 'PM'

sheet.merge_cells('K3:K4')
sheet['K3'].value = 'Assigned tester'

sheet.merge_cells('L3:N3')
sheet['L3'].value = 'Actual development time'
sheet['L4'].value = 'Start time'
sheet['M4'].value = 'Finish time'
sheet['N4'].value = 'Duration [h]'

sheet.merge_cells('O3:P3')
sheet['O3'].value = 'Commit statistics'
sheet['O4'].value = 'Insertions'
sheet['P4'].value = 'Deletions'

sheet.merge_cells('Q3:Q4')
sheet['Q3'].value = 'Estimated code documentation time [h]'

sheet.merge_cells('R3:R4')
sheet['R3'].value = 'Jira ID'

sheet.merge_cells('S3:S4')
sheet['S3'].value = 'Commit ID'

sheet.merge_cells('T3:T4')
sheet['T3'].value = 'Comment'


workbook.save(filename="hello_world.xlsx")

In [8]:
def get_schema_from_sheet(sheet): # TODO schema validation
    schema = {}
    schema['offset_row'] = 1 # offset of WBS: TODO search for WBS
    schema['offset_col'] = 0 # offset of WBS: TODO search for WBS
    # schema['header_1'] = [i.value for i in sheet['1']] # TODO apply offsets
    # schema['header_2'] = [i.value for i in sheet['2']] # TODO apply offsets
    # schema['map_to_jira'] = { # TODO something with history (start, finish, duration columns)
    #     'Name': 'summary',
    #     'Description': 'description',
    #     'Jira ID': 'key',
    # }
    schema['columns'] = { # TODO make it dynamic
        'WBS': 'A',
        'Name': 'B',
        'Description': 'C',
        'Jira ID': 'R',
    }
    schema['data_offset_row'] = 3
    return schema


def get_default_schema_from_template_xlsx(input_template_file = 'template.xlsx', output_schema_file = None):
    from openpyxl import load_workbook
    import json
    workbook = load_workbook(filename=input_template_file)
    sheet = workbook['Tasks']

    schema = get_schema_from_sheet(sheet)

    if output_schema_file == None:
        return schema
    else:
        json.dump(schema, open(output_schema_file, 'w'))


def get_issues_from_jira_and_create_xlsx_document_from_template(jira, output_file, project, label, input_template_file = 'template.xlsx'):
    from openpyxl import load_workbook
    import json
    workbook = load_workbook(filename=input_template_file)
    sheet = workbook['Tasks']

    schema = get_schema_from_sheet(sheet)

    issues = jira.jql('project=' + str(project), fields=list(jira_standard_fields.inv.keys()), limit=1000)
    # issues = jira.jql('project=' + str(project) + ' and labels=' + str(label), fields=list(jira_standard_fields.inv.keys()), limit=1000)

    # TODO


def xlsx_to_jira(jira, project, label, input_filename):
    from openpyxl import load_workbook
    
    workbook = load_workbook(filename=input_template_file)
    sheet = workbook['Tasks']

    schema = get_schema_from_sheet(sheet)
    

def wbs_regex_check(string, separator = '.'):
    import re
    return True if re.match('\A([0-9]+\\' + separator + '?)+\Z', string) else False

def wbs_regex_task(string):
    import re
    return True if re.match('\A[0-9]+\Z', string) else False

def get_jira_key_for_wbs(schema, wbs):
    i = schema['data_offset_row']
    while sheet[schema['columns']['WBS'] + str(i)].value is not None:
        if wbs == sheet[schema['columns']['WBS'] + str(i)].value:
            return sheet[schema['columns']['Jira ID'] + str(i)].value
        i = i + 1 
    return None

def create_or_update_issue(jira, schema, index, project, type, label, parent = None):
    if sheet[schema['columns']['Jira ID'] + str(index)].value is None: # create issue
        issue = jira.issue_create({
            'summary': sheet[schema['columns']['Name'] + str(index)].value,
            'description': sheet[schema['columns']['Description'] + str(index)].value,
            'project': {'key': project},
            'issuetype': {'name': type},
            'parent': {'key': parent},
            # 'Priority': 'priority',
            # 'Status': 'status',
            # 'Creator': 'creator',
            # 'Assignee': 'assignee',
        })
        sheet[schema['columns']['Jira ID'] + str(index)].value = issue['key']
        sheet[schema['columns']['Jira ID'] + str(index)].hyperlink = jira.url + '/browse/' + issue['key']
        sheet[schema['columns']['Jira ID'] + str(index)].font = Font(underline='single', color='0000FF')
        print(issue)
    else:
        jira.issue_update(sheet[schema['columns']['Jira ID'] + str(index)].value, {
            'summary': sheet[schema['columns']['Name'] + str(index)].value,
            'description': sheet[schema['columns']['Description'] + str(index)].value,
        })
    jira.edit_issue(issue_id_or_key=sheet[schema['columns']['Jira ID'] + str(index)].value, fields={"labels": [{"add": label}]}, notify_users=False)

def create_or_update_issues(jira, sheet, schema, project, label):
    i = schema['data_offset_row']
    while sheet.acell(schema['columns']['WBS'] + str(i)).value is not None:
        print(sheet.acell(schema['columns']['WBS'] + str(i)).value)
        # wbs = sheet[schema['columns']['WBS'] + str(i)].value
        # if not wbs_regex_check(wbs):
        #     raise Exception('Problem in WBS number structure: ' + str(wbs))

        # if wbs_regex_task(wbs):
        #     create_or_update_issue(jira=jira, schema=schema, index=i, project=project, type='Task', label=label)
        # else:
        #     import re
        #     parent = get_jira_key_for_wbs(schema=schema, wbs=re.match('\A[0-9]+', wbs)[0])
        #     if parent is None:
        #         raise Exception('Task doesn\'t have a parent: ' + wbs)
        #     create_or_update_issue(jira=jira, schema=schema, index=i, project=project, type='Sub-task', label=label, parent=parent)

        i = i + 1


In [4]:
import gspread

# google_info = {
#     'username': 'awaz@ownedoutcomes.com',
#     'password': '@rturW@Z11'
# }

# gc = gspread.login(google_info['username'], google_info['password'])

gc = gspread.oauth()

workbook = gc.open_by_key('1Vx7lAPI_i08nTesg1YpWaFxtmlvWqfyXgd3JWp9PfwY')
sheet = workbook.worksheet('Tasks')

In [ ]:
sheet